In [ ]:
import simpy
import random

# --- Simulation Parameters ---
STATION_1_TIME = 10       # Mins to process at Station 1
STATION_2_TIME = 20       # Mins to process at Station 2 (The Bottleneck)
STATION_3_TIME = 8        # Mins to process at Station 3
BUFFER_CAPACITY = 10      # Max units a conveyor buffer can hold
INTER_ARRIVAL_TIME = 12   # A new part arrives every 12 minutes on average
SIM_TIME = 1000           # Total simulation time in minutes

# A counter for parts that have successfully completed the line
parts_completed = 0

def part_process(env, part_name, station1, station2, station3, buffer12, buffer23):
    """
    This function simulates the journey of a single part through the assembly line.
    This is a 'generator' function required by SimPy.
    """
    global parts_completed
    print(f"{env.now:.2f}: {part_name} arrives at the assembly line.")

    # --- Process at Station 1 ---
    with station1.request() as req:
        yield req # Request one of the machines at the station
        print(f"{env.now:.2f}: {part_name} starts processing at Station 1.")
        yield env.timeout(STATION_1_TIME)
        print(f"{env.now:.2f}: {part_name} finishes at Station 1.")

    # --- Move to Buffer between Station 1 and 2 ---
    print(f"{env.now:.2f}: {part_name} attempts to move to Buffer 1->2.")
    yield buffer12.put(1) # This will automatically wait if the buffer is full
    # CORRECTED LINE
    print(f"{env.now:.2f}: {part_name} ENTERS Buffer 1->2. (Buffer level: {buffer12.level}/{buffer12.capacity})")

    # --- Process at Station 2 (The Bottleneck) ---
    with station2.request() as req:
        yield buffer12.get(1) # The part is now out of the buffer
        # CORRECTED LINE
        print(f"{env.now:.2f}: {part_name} EXITS Buffer 1->2, moves to Station 2. (Buffer level: {buffer12.level}/{buffer12.capacity})")
        yield req # Request the machine at Station 2
        print(f"{env.now:.2f}: {part_name} starts processing at Station 2.")
        yield env.timeout(STATION_2_TIME)
        print(f"{env.now:.2f}: {part_name} finishes at Station 2.")

    # --- Move to Buffer between Station 2 and 3 ---
    print(f"{env.now:.2f}: {part_name} attempts to move to Buffer 2->3.")
    yield buffer23.put(1)
    # CORRECTED LINE
    print(f"{env.now:.2f}: {part_name} ENTERS Buffer 2->3. (Buffer level: {buffer23.level}/{buffer23.capacity})")

    # --- Process at Station 3 ---
    with station3.request() as req:
        yield buffer23.get(1)
        # CORRECTED LINE
        print(f"{env.now:.2f}: {part_name} EXITS Buffer 2->3, moves to Station 3. (Buffer level: {buffer23.level}/{buffer23.capacity})")
        yield req
        print(f"{env.now:.2f}: {part_name} starts processing at Station 3.")
        yield env.timeout(STATION_3_TIME)
        print(f"{env.now:.2f}: {part_name} finishes at Station 3.")

    print(f"{env.now:.2f}: {part_name} has COMPLETED the line and exits.")
    parts_completed += 1


def part_source(env, station1, station2, station3, buffer12, buffer23):
    """
    This function generates new parts arriving at the assembly line.
    """
    part_id = 0
    while True:
        # Wait for a random time based on the average inter-arrival time
        yield env.timeout(random.expovariate(1.0 / INTER_ARRIVAL_TIME))
        part_id += 1
        # Start the journey for the new part
        env.process(part_process(env, f"Part-{part_id}", station1, station2, station3, buffer12, buffer23))


# --- Setup and Run the Simulation ---
print("--- Starting Assembly Line Simulation ---")
env = simpy.Environment()

# Define the resources for our simulation
# Stations are resources that can be used by one part at a time (capacity=1)
station1 = simpy.Resource(env, capacity=1)
station2 = simpy.Resource(env, capacity=1)
station3 = simpy.Resource(env, capacity=1)

# Buffers are containers that can hold a limited number of parts
buffer12 = simpy.Container(env, capacity=BUFFER_CAPACITY, init=0)
buffer23 = simpy.Container(env, capacity=BUFFER_CAPACITY, init=0)

# Start the part generator
env.process(part_source(env, station1, station2, station3, buffer12, buffer23))

# Run the simulation for the specified time
env.run(until=SIM_TIME)

# --- Simulation Report ---
print("\n--- Simulation Finished ---")
print(f"Total simulation time: {SIM_TIME} minutes.")
print(f"Total parts completed: {parts_completed}.")
throughput = parts_completed / (SIM_TIME / 60) # Units per hour
print(f"Throughput: {throughput:.2f} units per hour.")

--- Starting Assembly Line Simulation ---
4.23: Part-1 arrives at the assembly line.
4.23: Part-1 starts processing at Station 1.
14.23: Part-1 finishes at Station 1.
14.23: Part-1 attempts to move to Buffer 1->2.
14.23: Part-1 ENTERS Buffer 1->2. (Buffer level: 1/10)
14.23: Part-1 EXITS Buffer 1->2, moves to Station 2. (Buffer level: 0/10)
14.23: Part-1 starts processing at Station 2.
20.60: Part-2 arrives at the assembly line.
20.60: Part-2 starts processing at Station 1.
25.29: Part-3 arrives at the assembly line.
25.49: Part-4 arrives at the assembly line.
29.43: Part-5 arrives at the assembly line.
30.60: Part-2 finishes at Station 1.
30.60: Part-2 attempts to move to Buffer 1->2.
30.60: Part-2 ENTERS Buffer 1->2. (Buffer level: 1/10)
30.60: Part-3 starts processing at Station 1.
30.60: Part-2 EXITS Buffer 1->2, moves to Station 2. (Buffer level: 0/10)
34.23: Part-1 finishes at Station 2.
34.23: Part-1 attempts to move to Buffer 2->3.
34.23: Part-1 ENTERS Buffer 2->3. (Buffer leve

You should consider upgrading via the 'd:\rl-assembly-line-optimization\rl-assembly-line-optimization\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install simpy -q

import simpy
import random
import statistics
import numpy as np

# --- Simulation Parameters ---
BUFFER_CAPACITY = 10
INTER_ARRIVAL_TIME = 10
SIM_TIME = 10000 # Run for longer to see the effects of priorities and failures

# NEW: Parameters for the rework loop
FAIL_RATE = 0.10  # 8% of parts fail testing at Station 2
REPAIR_TIME = 30  # Time it takes to repair a failed part

# --- Part Configurations and Production Mix ---
PART_CONFIGS = {
    'Type_A': {'s1_time': 10, 's2_time': 20, 's3_time': 8},
    'Type_B': {'s1_time': 12, 's2_time': 25, 's3_time': 7},
    'Type_C': {'s1_time': 8,  's2_time': 18, 's3_time': 9},
}
PART_MIX = {'Type_A': 0.60, 'Type_B': 0.25, 'Type_C': 0.15}
PART_TYPES = list(PART_MIX.keys())
PART_PROBABILITIES = list(PART_MIX.values())

# NEW: Priority Mix (e.g., 20% HIGH priority, 80% LOW priority)
PRIORITY_MIX = {'HIGH': 0.20, 'LOW': 0.80}
# In SimPy, lower number = higher priority
PRIORITY_MAP = {'HIGH': 1, 'LOW': 2}


# --- Data Collection Structures ---
cycle_times_by_priority = {1: [], 2: []} # Store cycle times for HIGH and LOW priority parts separately
wip_log = []
station_busy_time = {'station1': 0, 'station2': 0, 'station3': 0, 'repair_station': 0}
parts_processed_per_station = {'station1': 0, 'station2': 0, 'station3': 0}


def part_process(env, part_name, part_priority, part_config, stations, buffers):
    arrival_time = env.now

    # --- Process at Station 1 ---
    # MODIFIED: Requesting with priority
    with stations['station1'].request(priority=part_priority) as req:
        yield req
        start_proc_time = env.now
        yield env.timeout(part_config['s1_time'])
        station_busy_time['station1'] += env.now - start_proc_time
        parts_processed_per_station['station1'] += 1

    yield buffers['buffer12'].put(1)

    # --- NEW: Test/Rework Loop for Station 2 ---
    tested_successfully = False
    while not tested_successfully:
        # --- Process at Station 2 (Testing) ---
        with stations['station2'].request(priority=part_priority) as req:
            yield req
            yield buffers['buffer12'].get(1)
            start_proc_time = env.now
            yield env.timeout(part_config['s2_time'])
            station_busy_time['station2'] += env.now - start_proc_time

        # --- Decision Point: Test Pass or Fail? ---
        if random.random() > FAIL_RATE:
            tested_successfully = True
            parts_processed_per_station['station2'] += 1 # Count only successful tests
            yield buffers['buffer23'].put(1)
        else:
            # Part failed, send to repair
            # print(f"{env.now:.2f}: {part_name} FAILED test. Rerouting to Repair.")
            with stations['repair_station'].request(priority=part_priority) as repair_req:
                yield repair_req
                start_repair_time = env.now
                yield env.timeout(REPAIR_TIME)
                station_busy_time['repair_station'] += env.now - start_repair_time

            # After repair, it needs to go back into the queue for Station 2
            yield buffers['buffer12'].put(1)

    # --- Process at Station 3 ---
    with stations['station3'].request(priority=part_priority) as req:
        yield req
        yield buffers['buffer23'].get(1)
        start_proc_time = env.now
        yield env.timeout(part_config['s3_time'])
        station_busy_time['station3'] += env.now - start_proc_time
        parts_processed_per_station['station3'] += 1

    finish_time = env.now
    cycle_times_by_priority[part_priority].append(finish_time - arrival_time)


def part_source(env, stations, buffers):
    part_id = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / INTER_ARRIVAL_TIME))
        part_id += 1

        # Select part type (for processing times)
        part_type = np.random.choice(PART_TYPES, p=PART_PROBABILITIES)
        part_config = PART_CONFIGS[part_type]

        # NEW: Select part priority
        priority_name = np.random.choice(list(PRIORITY_MIX.keys()), p=list(PRIORITY_MIX.values()))
        part_priority = PRIORITY_MAP[priority_name]

        part_name = f"Part-{part_id}({part_type}, P{part_priority})"
        env.process(part_process(env, part_name, part_priority, part_config, stations, buffers))


def monitor_wip(env, buffers):
    while True:
        wip_log.append((buffers['buffer12'].level, buffers['buffer23'].level))
        yield env.timeout(10) # Monitor less frequently for long simulations

# --- Setup and Run the Simulation ---
print("--- Starting Full-Featured Simulation (Priorities & Rework) ---")
env = simpy.Environment()

# MODIFIED: All stations are now PriorityResources
stations = {
    'station1': simpy.PriorityResource(env, capacity=1),
    'station2': simpy.PriorityResource(env, capacity=1),
    'station3': simpy.PriorityResource(env, capacity=1),
    'repair_station': simpy.PriorityResource(env, capacity=1)
}
buffers = {
    'buffer12': simpy.Container(env, capacity=BUFFER_CAPACITY, init=0),
    'buffer23': simpy.Container(env, capacity=BUFFER_CAPACITY, init=0)
}

env.process(part_source(env, stations, buffers))
env.process(monitor_wip(env, buffers))
env.run(until=SIM_TIME)

# --- KPI Calculation and Reporting ---
print("\n--- Simulation Finished ---")
print(f"Total simulation time: {SIM_TIME} minutes.")

# NEW: Reporting the simulation parameters used for this run
print(f"\n--- Simulation Parameters ---")
print("Production Mix:")
for part_type, percentage in PART_MIX.items():
    print(f"  - {part_type}: {percentage:.0%}")
print("Priority Mix:")
for priority_name, percentage in PRIORITY_MIX.items():
    print(f"  - {priority_name} Priority: {percentage:.0%}")
print(f"Station 2 Fail Rate: {FAIL_RATE:.0%}")


# --- Throughput and Cycle Time ---
high_prio_parts_completed = len(cycle_times_by_priority[1])
low_prio_parts_completed = len(cycle_times_by_priority[2])
total_parts_completed = high_prio_parts_completed + low_prio_parts_completed
throughput_per_hour = total_parts_completed / (SIM_TIME / 60)

print(f"\n--- Primary KPI: Throughput ---")
print(f"Total parts completed: {total_parts_completed} (HIGH: {high_prio_parts_completed}, LOW: {low_prio_parts_completed})")
print(f"Overall Line Throughput: {throughput_per_hour:.2f} units per hour")

avg_cycle_time_high = statistics.mean(cycle_times_by_priority[1]) if high_prio_parts_completed > 0 else 0
avg_cycle_time_low = statistics.mean(cycle_times_by_priority[2]) if low_prio_parts_completed > 0 else 0

print(f"\n--- Secondary KPI: Cycle Time by Priority ---")
print(f"Average Cycle Time (HIGH Prio): {avg_cycle_time_high:.2f} minutes")
print(f"Average Cycle Time (LOW Prio):  {avg_cycle_time_low:.2f} minutes")

# --- WIP ---
final_wip_12 = buffers['buffer12'].level
final_wip_23 = buffers['buffer23'].level
print(f"\n--- Secondary KPI: Work-In-Progress (WIP) ---")
print(f"Final WIP Count in Buffer 1->2: {final_wip_12} units")
print(f"Final WIP Count in Buffer 2->3: {final_wip_23} units")

# --- Utilization ---
print(f"\n--- Secondary KPI: Station Utilization ---")
for name, resource in stations.items():
    utilization = (station_busy_time[name] / SIM_TIME) * 100
    print(f"{name.replace('_', ' ').title()} Utilization: {utilization:.2f}%")

--- Starting Full-Featured Simulation (Priorities & Rework) ---

--- Simulation Finished ---
Total simulation time: 10000 minutes.

--- Simulation Parameters ---
Production Mix:
  - Type_A: 60%
  - Type_B: 25%
  - Type_C: 15%
Priority Mix:
  - HIGH Priority: 20%
  - LOW Priority: 80%
Station 2 Fail Rate: 10%

--- Primary KPI: Throughput ---
Total parts completed: 425 (HIGH: 92, LOW: 333)
Overall Line Throughput: 2.55 units per hour

--- Secondary KPI: Cycle Time by Priority ---
Average Cycle Time (HIGH Prio): 2658.24 minutes
Average Cycle Time (LOW Prio):  2946.52 minutes

--- Secondary KPI: Work-In-Progress (WIP) ---
Final WIP Count in Buffer 1->2: 10 units
Final WIP Count in Buffer 2->3: 0 units

--- Secondary KPI: Station Utilization ---
Station1 Utilization: 99.40%
Station2 Utilization: 99.64%
Station3 Utilization: 33.45%
Repair Station Utilization: 14.70%
